In [ ]:
def crop(image,input_size):# 이미지 하나, 모델에 들어갈 사이즈 / dlib.get_frontal_face_detector() 모델을 이용해 얼굴을 찾고 잘라낸다.
    imgs=[]
    detector = dlib.get_frontal_face_detector()
    faces = detector(image)
    for face in faces:
        face_img=image[face.top():face.bottom(), face.left():face.right()]
        face_img=cv2.resize(face_img,dsize=input_size)
        face_input=face_img.copy().reshape((input_size[1], input_size[0], 3)).astype(np.float32)/255.
        imgs.append(face_input)
    return imgs# 얼굴 여러개

In [ ]:
def load_imgs(img_path):# 모델 폴더 경로를 받아서 내부의 jpg 파일을 모두 로드한다.
    result=[]
    imgs_path_list= [x for x in glob.glob(img_path + '/*.jpg')]

    for img_path in imgs_path_list:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img,fx=1, fy=1,dsize=(0,0),interpolation=cv2.INTER_AREA)
        result.append(img)
    return result # 리스트 형태로 여러장 이미지 반환

In [ ]:
def rotate(imgs,shapes):# dlib.shape_predictor 모델을 이용해 얻은 쉐입 정보로 코사인 법칙을 이용해 각도를 구하고 회전 행렬을 구해 회전시킨다.
    rotated_imgs = []
    for i,shape in enumerate(shapes):
        cos,direction =trig(shape[39],shape[42])
        radians= np.arccos(cos)
        angle = 90 - (radians*180/math.pi)
        (h, w) = imgs[i].shape[:2]
        (cx, cy) = (w // 2, h // 2)
        rotate_matrix = cv2.getRotationMatrix2D((cx, cy), angle*direction, 1.0)
        rotate_img= cv2.warpAffine(np.array(imgs[i]), rotate_matrix, (w, h))
        rotated_imgs.append(rotate_img)
    return rotated_imgs   

In [ ]:
def shape_to_np(shape, dtype="int"):# dlib.shape_predictor에서 얻은 정보를 실제적으로 사용가능하게 해주는 함수
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [ ]:
def shape(imgs,predictor_path):# 얼굴 쉐입을 찾고 리스트로 저장해주는 함수
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    shape_list = []
    for img in imgs:
        faces=detector(img)
        for face in faces:
            shape= predictor(img,face)
            shape= shape_to_np(shape)
            shape_list.append(shape)
    return shape_list

In [ ]:
def trig(point_1,point_2):# cp가 1이면 반시계방향 회전 -1이면 시계 방향 회전 / 코사인 법칙을 이용해 방향과 cos값 구함
    x1=point_1[0]
    y1=point_1[1]
    x2=point_2[0]
    y2=point_2[1]

    a=abs((x2-x1))
    b=abs((y2-y1))
    c= math.sqrt(math.pow(a,2)+math.pow(b,2))

    cos=(math.pow(b,2)+math.pow(c,2)-math.pow(a,2))/(2*b*c)

    if y1>=y2:
        direction= -1
    else:
        direction= 1

    return cos,direction

In [ ]:
profile_images_path='./images/profile' # 프로필 이미지 경로/ 여러장 가능 
selfie_image_path= './images/selfie'# 셀프 카메라 이미지 경로 / 한장만 가능 
model_path= './model/20180402-114759.pb'
predictor_path ='./model/shape_predictor_68_face_landmarks.dat'


input_size= (160,160)# 모델에 삽입되는 사이즈 / 변경하면 모델 안 돌아감 


##-------------------------------------------------Load and crop images --------------------------------------------------------------------##

profile_imgs = load_imgs(profile_images_path) # 1번에서 받은 프로필 이미지 로드
selfie_img =load_imgs(selfie_image_path)# 2번에서 받은 프로필 이미지 로드

predictor = dlib.shape_predictor(predictor_path) # 얼굴의 쉐입을 찾는 모델 선언
detector = dlib.get_frontal_face_detector() # 얼굴 찾아주는 모델 선언

profile_faces=[]
for img in profile_imgs:
    detected_faces=crop(img,input_size)# 모델을 통해 찾은 얼굴을 자르고 모델에 삽입되는 사이즈로 변경한다
    for detected_face in detected_faces:
        profile_faces.append(detected_face)# 얼굴 하나씩 들어있다.

selfie_faces=crop(selfie_img[0],input_size) # 모델을 통해 찾은 얼굴을 자르고 모델에 삽입되는 사이즈로 변경한다


# shape 리스트 생성


profile_shapes= shape(profile_imgs,predictor_path) # 12번에서 받은 프로필 이미지에서 얼굴을 찾고, 찾은 얼굴에 쉐입을 찾는다.

selfie_shapes = shape(selfie_img,predictor_path) # 13번에서 받은 프로필 이미지에서 얼굴을 찾고, 찾은 얼굴에 쉐입을 찾는다.


# shape 정보를 바탕으로 회전

profile_rotated_imgs=rotate(profile_faces,profile_shapes) # 프로필 사진 얻은 쉐입 정보를 바탕으로 회전시킨다.
selfie_rotated_imgs=rotate(selfie_faces,selfie_shapes)# 프로필 사진 얻은 쉐입 정보를 바탕으로 회전시킨다.




# ##--------------------------------------------------------prediction--------------------------------------------------------------##
facenet= Facenet(model_path)

print("profile에서 탐지된 얼굴 수 :",len(profile_faces))
print("selfie에서 탐지된 얼굴 수 :",len(selfie_faces))

profile_predictions= facenet.get_embeddings(profile_rotated_imgs)# 회전된 얼굴들을 벡터로 임베딩한다 
selfie_predictions = facenet.get_embeddings(selfie_rotated_imgs) #회전된 얼굴들을 벡터로 임베딩한다 

profile_predictions=np.reshape(profile_predictions,[-1,1,512])
selfie_predictions=np.reshape(selfie_predictions,[-1,1,512])


eucledian_dist = []
print("벡터 거리 정보/ 0.9 이하는 동일인")
for i in range(len(profile_predictions)):
    for j in range(len(selfie_predictions)):
        dist=np.linalg.norm(profile_predictions[i]-selfie_predictions[j])
        eucledian_dist.append(dist)
print(eucledian_dist)



## ----------------------------------- 이미지 체크할 때/ loded_imgs에 profile_imgs 나 selfie_imgs 를 써서 크로핑 된 이미지를 확인 가능하다 -------------------------------## 
loaded_imgs = profile_rotated_imgs# select profile_imgs or selfie_imgs


for i in loaded_imgs:
    a= cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    a=np.array(a)
    plt.imshow(a)
    plt.show()
